In [ ]:
!pip install tensorflow==2.17.0 keras==3.4.1

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Embedding,Input

In [ ]:
max_features = 10000
input_shape = (500,)
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)
print(X_train.shape, y_train.shape)

(25000,) (25000,)


In [ ]:
max_len=1000
X_train = sequence.pad_sequences(X_train, maxlen=max_len)
X_test = sequence.pad_sequences(X_test, maxlen=max_len)
print(X_train.shape, X_test.shape)

(25000, 1000) (25000, 1000)


In [ ]:
model=Sequential()
model.add(Input(shape=(max_len,)))
model.add(Embedding(max_features,128))
model.add(SimpleRNN(128,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (None, 1000, 128)           │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_2 (SimpleRNN)             │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,313,025 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
earlystopping=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)
earlystopping

In [ ]:
history=model.fit(
    X_train,y_train,epochs=10,batch_size=32,
    validation_split=0.2,
    callbacks=[earlystopping]
)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 73s 113ms/step - accuracy: 0.5836 - loss: 7.4745 - val_accuracy: 0.6534 - val_loss: 0.5969
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 80s 111ms/step - accuracy: 0.7596 - loss: 0.5148 - val_accuracy: 0.7854 - val_loss: 0.4670
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 82s 112ms/step - accuracy: 0.8537 - loss: 0.3473 - val_accuracy: 0.8080 - val_loss: 0.4302
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 82s 111ms/step - accuracy: 0.9077 - loss: 0.2397 - val_accuracy: 0.8230 - val_loss: 0.4329
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 82s 111ms/step - accuracy: 0.7783 - loss: 41535240.0000 - val_accuracy: 0.6734 - val_loss: 0.5989
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 73s 117ms/step - accuracy: 0.7758 - loss: 0.5273 - val_accuracy: 0.6984 - val_loss: 0.5738
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 78s 111ms/step - accuracy: 0.8171 - loss: 0.4789 - val_accuracy: 0.7158 - val_loss: 0.5538
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 70s 111ms/step - accuracy: 0.8430 - 

In [ ]:
model.save('simple_rnn_imdb.keras')
model.save('simple_rnn_imdb(1).h5')

In [ ]:
word_index = imdb.get_word_index()
reverse_word_index = {value: key for key, value in word_index.items()}

1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_review])

# Function to preprocess user input
def preprocess_text(text):
    words = text.lower().split()
    encoded_review = [word_index.get(word, 2) + 3 for word in words]
    padded_review = sequence.pad_sequences([encoded_review], maxlen=500)
    return padded_review

In [ ]:
def predict_sentiment(review):
    preprocessed_input=preprocess_text(review)

    prediction=model.predict(preprocessed_input)

    sentiment = 'Positive' if prediction[0][0] > 0.5 else 'Negative'

    return sentiment, prediction[0][0]

In [ ]:
review="This movie was worst"
sentiment,score=predict_sentiment(review)
print(f'Review: {review}')
print(f'Sentiment: {sentiment}')
print(f'Prediction Score: {score}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Review: This movie was worst
Sentiment: Positive
Prediction Score: 0.9835739135742188
